In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

In [3]:
# create a density matrix of size 2 usinf Fock base preparation

RHO0 = fock_dm(2)

print(RHO0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]


In [12]:
# building a Hamiltoninan using 
H = 2*np.pi * 0.1 * sigmax()

In [13]:

# definition os the ket of dimension 2 : basis(2, 0)
psi0 = basis(2, 0)

times = np.linspace(0.0, 10.0, 100)

result = mesolve(H, psi0, times, [np.sqrt(0.05) * sigmax()], [sigmaz(), sigmay()])

fig, ax = plt.subplots()

ax.plot(times, result.expect[0]) 

ax.plot(times, result.expect[1]) 

ax.set_xlabel('Time') 

ax.set_ylabel('Expectation values') 

ax.legend(("Sigma-Z", "Sigma-Y"))  

plt.show() 

ValueError: dimension mismatch